# Interactive use

This notebook includes an interactive demonstration

In [ ]:
%pip install svg.py numpy matplotlib pandas ipywidgets

In [1]:
from lmunplugged import Bin,Table, Ball, Sticky, Doc, TrainDemo
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

The next cell will create an interactive interface you can use to experiment with the training process. It does not yet handle the end document well, it will cycle back to the beginning if you press train an additional time after it adds the white ball. 

In [ ]:
# TODO: fix this demo to use Doc.is_valid and to disable buttons at end of doc instead of resetting
initital_doc = ['green','blue','purple','blue','pink','blue','white']
bin_colors = ['purple','blue','green','pink']
allowed_colors = bin_colors + ['white']
color_tags = widgets.ColorsInput(
    value=initital_doc,
    allowed_tags=allowed_colors,
    # allow_duplicates=False
)

my_doc = Doc.from_list(color_tags.value,max_width_words=5)
to_train = Table.from_list(bin_colors)

my_demo = TrainDemo(to_train, my_doc)

html_output_demo= widgets.Output()
with html_output_demo:
    display(my_demo)

def set_demo_button(b):
    my_demo.doc.reset_words(color_tags.value)
    my_demo.reset_training()
    with html_output_demo:
        html_output_demo.clear_output(wait=True)
        display(my_demo)


def train_step(b):    
    with html_output_demo:
        html_output_demo.clear_output(wait=True)
        display(my_demo.train_next())



demo_button = widgets.Button(
    description="load doc",
    button_style='info', 
    layout=widgets.Layout(width='100px', height='30px')
)

step_button = widgets.Button(
    description="step",
    button_style='success', 
    layout=widgets.Layout(width='100px', height='30px')
)

demo_button.on_click(set_demo_button)
step_button.on_click(train_step)
demo_ctrl = widgets.HBox([ demo_button,step_button])
layout = widgets.VBox([color_tags,demo_ctrl,html_output_demo])

In [ ]:
display(layout)